In [0]:
bronze_df = spark.table("workspace.healthcare.bronze_claims_raw")

# Null profile
display(
    bronze_df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c+"_nulls")
        for c in bronze_df.columns
    ])
)

# Deduplicate
from pyspark.sql.window import Window
w = Window.partitionBy("ClaimID").orderBy(F.col("ingest_ts").desc())
deduped = (bronze_df
           .withColumn("rn", F.row_number().over(w))
           .filter("rn=1")
           .drop("rn"))

display(deduped.limit(5))